### Get results from Wandb

In [1]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("draftrec/atari_pretrain")

summary_list, config_list, id_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    id_list.append(run.id)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "id": id_list,
    })

runs_df.to_csv("project.csv")

### Read runs

In [2]:
data_ = pd.read_csv('project.csv', index_col=0)
data_

,summary,config,id
0,{'_wandb': {'runtime': 261}},{},2r66xrkx
1,{'_wandb': {'runtime': 6}},{},168znuio
2,{},{},3laam0jm
3,{},{},2xxbk2rt
4,{'_wandb': {'runtime': 1}},"{'exp_name': 'gpt_video_cons_reg', 'overrides'...",1sejcyvj
...,...,...,...
1509,"{'pos_neg_diff': 0.87976934150327, 'positive_s...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",5zlpym7v
1510,"{'positive_sim': 0.998840137720108, 'loss': -0...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",3oh4f4sw
1511,"{'_wandb': {'runtime': 18405}, '_runtime': 183...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",385p3sw7
1512,"{'_step': 117001, '_wandb': {'runtime': 17734}...","{'env': {'game': 'pong', 'type': 'atari', 'hor...",njf5gwab


### Filter based on conditions

In [3]:
group_name = 'baseline'
exp_name = 'gpt_video_cons_reg'
model_path = '0/10/model.pth'

In [4]:
configs = data_['config']
indexs = []
for idx, config in enumerate(configs):
    cfg = eval(config)
    if len(cfg) == 0:
        continue
        
    run_exp_name = cfg['exp_name']
    run_group_name = cfg['group_name']

    # condition
    if run_exp_name == exp_name and run_group_name == group_name:
        if 'env' in cfg:
            indexs.append(idx)

In [5]:
data = data_.iloc[indexs]
envs = []
for config in data['config']:
    envs.append(eval(config)['env']['game'])
data['env'] = envs
print(len(data))
data

26


/tmp/ipykernel_142072/3806693401.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['env'] = envs


,summary,config,id,env
116,"{'pos_neg_diff': 0.4865609109401703, 'reward_r...","{'env': {'game': 'PrivateEye', 'seed': 42, 'ty...",22h0p5n2,PrivateEye
117,"{'_wandb': {'runtime': 169671}, '_runtime': 16...","{'env': {'game': 'Qbert', 'seed': 42, 'type': ...",2g14o9bn,Qbert
118,"{'_runtime': 168503, 'rank_eps1': 511, 'max_gr...","{'env': {'game': 'KungFuMaster', 'seed': 42, '...",2n5s4p1a,KungFuMaster
119,"{'neg_sim': 0.5361865162849426, 'reg_loss': 2....","{'env': {'game': 'Kangaroo', 'seed': 42, 'type...",2pf1jz0s,Kangaroo
120,"{'act_acc': 0.059685616215422656, '_timestamp'...","{'env': {'game': 'Krull', 'seed': 42, 'type': ...",2rkly9cp,Krull
121,"{'rank_eps1': 494, 'rank_eps01': 512, 'rank_ep...","{'env': {'game': 'MsPacman', 'seed': 42, 'type...",1ngd6rri,MsPacman
122,"{'loss': 0.12374835543816672, '_runtime': 1656...","{'env': {'game': 'Pong', 'seed': 42, 'type': '...",35xml4ko,Pong
123,"{'act_acc': 0.027741896223601552, 'act_loss': ...","{'env': {'game': 'RoadRunner', 'seed': 42, 'ty...",o05bspxw,RoadRunner
248,"{'loss': 0.03718355287154184, '_step': 937480,...","{'env': {'game': 'Frostbite', 'seed': 42, 'typ...",enaak6qz,Frostbite
249,"{'reward_ratio': 0.010350517525876294, '_wandb...","{'env': {'game': 'Jamesbond', 'seed': 42, 'typ...",1a13oqxs,Jamesbond


### 1. Restore Saved Models

In [8]:
import pathlib
base_path = str(pathlib.Path().resolve())

#artifact_dict = {}
model_path = '0/40/model.pth'
for run_id, env in zip(data['id'], data['env']):
    print(env, run_id)
    try:
        name = env + '/' + model_path 
        path = base_path + '/' + name
        wandb.restore(name, run_path="draftrec/atari_pretrain/" + run_id)
        artifact_dict[path] = name 
    except:
        continue

PrivateEye 22h0p5n2


wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.


Qbert 2g14o9bn
KungFuMaster 2n5s4p1a
Kangaroo 2pf1jz0s
Krull 2rkly9cp
MsPacman 1ngd6rri
Pong 35xml4ko
RoadRunner o05bspxw
Frostbite enaak6qz


wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.


Jamesbond 1a13oqxs
Freeway 22o38zo3
Gopher 2u9q47bo
ChopperCommand 30zwscqw
DemonAttack 3kemil59
CrazyClimber eztm8uxo
Hero 2s85gj42
UpNDown 3g4odmpc
Seaquest m2hlxny7
BankHeist 134iip6d
Assault 33zs3c6s
Boxing 3bsvfpvu
Alien 3jls2xx6
Amidar srtltnlq
Breakout 2dqyehv1
Asterix 331j6wki
BattleZone 6z9xqgxh


### Save to artifact

In [10]:
wandb.init(project='atari_pretrain',
           entity='draftrec',
           group=exp_name,
           settings=wandb.Settings(start_method="thread"))  
artifact = wandb.Artifact(name=exp_name, type='model')

# save models
for path, name in artifact_dict.items():
    artifact.add_file(path, name=name)

wandb.run.finish_artifact(artifact)

wandb: Currently logged in as: joonleesky (draftrec). Use `wandb login --relogin` to force relogin
/opt/conda/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


### Remove artifact

In [11]:
import shutil
for env in data['env']:
    shutil.rmtree('./'+env, ignore_errors=True)

### 2. Generate json file

In [ ]:
def get_results_dict(data):
    results = []
    for idx in range(len(data)):
        row = data.iloc[idx]
        summary = eval(row['summary'])
        config = eval(row['config'])

        if 'env' not in config:
            continue

        game = config['env']['game']
        try:
            import pdb
            pdb.set_trace()
            score = summary['eval_mean_traj_game_scores']
        except:
            continue
        results.append([0, game, score, 0])

    return results
    
results = get_results_dict(data)

In [ ]:
data

In [18]:
import json

def generate_json_file(file_name, results):
    data = {}
    data['data'] = results
    path = file_name + '.json'
    with open(path, 'w') as json_file:
        json.dump(data, json_file)

In [19]:
generate_json_file('byol_impala', results)